In [1]:
from crystalbuilder import *
import crystalbuilder.geometry as geo
import matplotlib.pyplot as plt
import numpy as np
import vedo

This notebook will walk through an example of creating a 3D photonic crystal based on the rod-connected diamond motif. The structure will be made using CrystalBuilder's geo package and then simulated using MPB. 

First, let's define our basis. This time, we will use a cubic basis and not the primitive rhombohedral one. 

In [2]:
a1 = [1,0, 0]
a2 = [.5,np.sqrt(3)/2 ,0]
a3 = [0, 0, 1]

a_mag = 1

geo_lattice = lattice.Lattice(a1, a2, a3, magnitude = [a_mag, a_mag, a_mag])

We've now defined our lattice. It's important to note that this is a CrystalBuilder lattice and not an MPB one. They can be converted, as we will show later, but they are not the exact same. 

Let's get the positions of the "atoms" in the diamond lattice that our rods will connect.

The square diamond lattice looks like 4 tetrahedra connected to one another. There are 18 sites in the cubic unit cell. 

To avoid manually specifying those 18 sites, let's take advantage of the Bilbao Crystallographic Server. Fortunately, CrystalBuilder includes a package that can grab and parse the generators from [the Bilbao site](https://www.cryst.ehu.es). 
<br>
<summary>The Bilbao database is summarized in the following papers, which should be cited if this function is used. </summary>

<details>

>
>M. I. Aroyo, J. M. Perez-Mato, D. Orobengoa, E. Tasci, G. de la Flor, A. Kirov
"Crystallography online: Bilbao Crystallographic Server"
Bulg. Chem. Commun. 43(2) 183-197 (2011).
>
>M. I. Aroyo, J. M. Perez-Mato, C. Capillas, E. Kroumova, S. Ivantchev, G. Madariaga, A. Kirov & H. Wondratschek
"Bilbao Crystallographic Server I: Databases and crystallographic computing programs"
 Z. Krist. 221, 1, 15-27 (2006)
>
>M. I. Aroyo, A. Kirov, C. Capillas, J. M. Perez-Mato & H. Wondratschek
"Bilbao Crystallographic Server II: Representations of crystallographic point groups and space groups"
Acta Cryst. A62, 115-128 (2006)
>
</details>

The bilbao.SpaceGroup() class contains functions for applying symmetry operations to specified points. An instance of the space group must be initialized, with the only argument being the IUCr space group number (1-230).

After this, the calculate_points function will accept any number of input coordinates and perform all of the symmetry operations. The output will be either a numpy array (default) or a list. If using a numpy array, the output will contain only the unique results of the symmetry operations. A list will contain duplicates, as np.unique() requires a conversion to an ndarray. For this reason, it is strongly recommended to use the default numpy output.

In [11]:
diamond = bilbao.SpaceGroup(182)

points = diamond.calculate_points(point_list=[(1/2, 0, 1/2)]) #This is the 8a wyckoff position

For a diamond lattice, the rods connect nearest-neighbor points. Using geo.NearestNeighbors, we can create cylinders between the points that are within a neighborhood_range of each other. The output is a list of geometries, which could be converted into a supercell if desired. 

<summary> Under the hood: </summary>
<details>
The details of this step involve creating a KDTree from the points, and calculating their distances in Euclidean space (Minkowski order 2). The neighborhood_range is in absolute coordinates and is not scaled to the lattice constant.
</details> 

In [12]:
radius = .25
# connection_sites = geo.NearestNeighbors(points, radius=radius, neighborhood_range=.45)

sphere_caps = [geo.Sphere(center=point, radius=radius) for point in points]
print(len(sphere_caps))
# geometry = [connection_sites] + sphere_caps

12


Now lets use our previously defined lattice to tile these cylinders into a crystal.

In [13]:
a1_reps = 1
a2_reps = 1
a3_reps = 1
crystal = geo_lattice.tile_geogeometry(sphere_caps, a1_reps, a2_reps, a3_reps )

We can visualize this using CrystalBuilder's viewer package, which builds the structure as a Vedo scene. 

In [14]:
scene = viewer.visualize(crystal)
scene.show().close()
objects = scene.get_meshes()

k=0
string = f"square_diamond/group-object.obj"
# vedo.write_group(objects, string)
for obj in objects:
    k+=1
    string=f"182/object-{k}.obj"
    vedo.write(obj, string)
